# Title goes here

**Author:** Elimelech Berlin  
**Date:** March 2023
***

### Overview
This report analyzes available data to identify home features that have a strong correlation with home sale prices. Included in this data is information about home features such as living space, utilities ...
***

### Business Problem
Investors looking to turn a profit from flipping houses are presented with the following dilemma: With a finite amount of funds to invest per property, how should they spend their money to maximize their return? What home features are associated with an increase in home value?  
In this report I explore a given dataset to pinpoint what sort of renovations/improvements investors would be advised to spend their money on in an effort to maximize a given home's resail value.
***

### Data
The data examined in this report describes properties located in King County, Washington and comes from the King County Assessor Data Download available from the [King County Department of Assesments webpage](https://info.kingcounty.gov/assessor/DataDownload/default.aspx). (Some of the properties are located outside of the county proper).  
This dataset contains information about a variety of home/property features of about 30,000 properties. Features described include general details (location, view from the home etc.) as well as details about the actual structure and layout (square-footage, utilities etc.) of the properties.  
In creating this report the following two files were used:
* [kc house data.csv](https://github.com/terminalcoder/dsc-phase-2-project-v2-5/blob/main/data/kc_house_data.csv) contains the dataset 
* [column names.md](https://github.com/terminalcoder/dsc-phase-2-project-v2-5/blob/main/data/column_names.md) contains descriptive information about each column in the dataset
***

In [30]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm

To get started, have a look at the data:

In [31]:
# load dataset from file & preview it
kc_house_data = pd.read_csv('data/kc_house_data.csv')
kc_house_data

id        date      price  bedrooms  bathrooms  sqft_living  \
0      7399300360   5/24/2022   675000.0         4        1.0         1180   
1      8910500230  12/13/2021   920000.0         5        2.5         2770   
2      1180000275   9/29/2021   311000.0         6        2.0         2880   
3      1604601802  12/14/2021   775000.0         3        3.0         2160   
4      8562780790   8/24/2021   592500.0         2        2.0         1120   
...           ...         ...        ...       ...        ...          ...   
30150  7834800180  11/30/2021  1555000.0         5        2.0         1910   
30151   194000695   6/16/2021  1313000.0         3        2.0         2020   
30152  7960100080   5/27/2022   800000.0         3        2.0         1620   
30153  2781280080   2/24/2022   775000.0         3        2.5         2570   
30154  9557800100   4/29/2022   500000.0         3        1.5         1200   

       sqft_lot  floors waterfront greenbelt  ... sewer_system sqft_above  \
0          7140     1.0         NO        NO  ...       PUBLIC       1180   
1          6703     1.0         NO        NO  ...       PUBLIC       1570   
2          6156     1.0         NO        NO  ...       PUBLIC       1580   
3          1400     2.0         NO        NO  ...       PUBLIC       1090   
4           758     2.0         NO        NO  ...       PUBLIC       1120   
...         ...     ...        ...       ...  ...          ...        ...   
30150      4000     1.5         NO        NO  ...       PUBLIC       1600   
30151      5800     2.0         NO        NO  ...       PUBLIC       2020   
30152      3600     1.0         NO        NO  ...       PUBLIC        940   
30153      2889     2.0         NO        NO  ...       PUBLIC       1830   
30154     11058     1.0         NO        NO  ...       PUBLIC       1200   

      sqft_basement sqft_garage sqft_patio yr_built  yr_renovated  \
0                 0           0         40     1969             0   
1              1570           0        240     1950             0   
2              1580           0          0     1956             0   
3              1070         200        270     2010             0   
4               550         550         30     2012             0   
...             ...         ...        ...      ...           ...   
30150          1130           0        210     1921             0   
30151             0           0        520     2011             0   
30152           920         240        110     1995             0   
30153           740         480        100     2006             0   
30154             0         420          0     1965             0   

                                                 address        lat       long  
0      2102 Southeast 21st Court, Renton, Washington ...  47.461975 -122.19052  
1      11231 Greenwood Avenue North, Seattle, Washing...  47.711525 -122.35591  
2      8504 South 113th Street, Seattle, Washington 9...  47.502045 -122.22520  
3      4079 Letitia Avenue South, Seattle, Washington...  47.566110 -122.29020  
4      2193 Northwest Talus Drive, Issaquah, Washingt...  47.532470 -122.07188  
...                                                  ...        ...        ...  
30150  4673 Eastern Avenue North, Seattle, Washington...  47.664740 -122.32940  
30151  4131 44th Avenue Southwest, Seattle, Washingto...  47.565610 -122.38851  
30152  910 Martin Luther King Jr Way, Seattle, Washin...  47.610395 -122.29585  
30153  17127 114th Avenue Southeast, Renton, Washingt...  47.449490 -122.18908  
30154  18615 7th Avenue South, Burien, Washington 981...  47.435840 -122.32634  

[30155 rows x 25 columns]

#### Data Preparation
Neaten the data to prepare for analysis

In [32]:
# create list of columns to ignore (drop) in this report
col_to_ignore = ['date', 'view', 'sqft_above', 'address', 'lat', 'long']

# drop those columns
kc_house_data.drop(col_to_ignore, axis=1, inplace=True)

In [33]:
# get a sense of the distributios of the features
kc_house_data.describe()

id         price      bedrooms     bathrooms   sqft_living  \
count  3.015500e+04  3.015500e+04  30155.000000  30155.000000  30155.000000   
mean   4.538104e+09  1.108536e+06      3.413530      2.334737   2112.424739   
std    2.882587e+09  8.963857e+05      0.981612      0.889556    974.044318   
min    1.000055e+06  2.736000e+04      0.000000      0.000000      3.000000   
25%    2.064175e+09  6.480000e+05      3.000000      2.000000   1420.000000   
50%    3.874011e+09  8.600000e+05      3.000000      2.500000   1920.000000   
75%    7.287100e+09  1.300000e+06      4.000000      3.000000   2619.500000   
max    9.904000e+09  3.075000e+07     13.000000     10.500000  15360.000000   

           sqft_lot        floors  sqft_basement   sqft_garage    sqft_patio  \
count  3.015500e+04  30155.000000   30155.000000  30155.000000  30155.000000   
mean   1.672360e+04      1.543492     476.039396    330.211142    217.412038   
std    6.038260e+04      0.567717     579.631302    285.770536    245.302792   
min    4.020000e+02      1.000000       0.000000      0.000000      0.000000   
25%    4.850000e+03      1.000000       0.000000      0.000000     40.000000   
50%    7.480000e+03      1.500000       0.000000    400.000000    150.000000   
75%    1.057900e+04      2.000000     940.000000    510.000000    320.000000   
max    3.253932e+06      4.000000    8020.000000   3580.000000   4370.000000   

           yr_built  yr_renovated  
count  30155.000000  30155.000000  
mean    1975.163953     90.922301  
std       32.067362    416.473038  
min     1900.000000      0.000000  
25%     1953.000000      0.000000  
50%     1977.000000      0.000000  
75%     2003.000000      0.000000  
max     2022.000000   2022.000000

In [34]:
# check for null values
kc_house_data.isna().sum()

id                0
price             0
bedrooms          0
bathrooms         0
sqft_living       0
sqft_lot          0
floors            0
waterfront        0
greenbelt         0
nuisance          0
condition         0
grade             0
heat_source      32
sewer_system     14
sqft_basement     0
sqft_garage       0
sqft_patio        0
yr_built          0
yr_renovated      0
dtype: int64

In [35]:
# drop rows with null values for ease of use
kc_house_data.dropna(inplace=True)

### Data Analysis

As this report aims to identify ways in which properties can be renovated or improved to maximize resale value, I will examine the data using the home features other than price as predictor variables and the target variable will be price.

First, get a sense of how each variable is independently correlated with the target variable:

In [36]:
# check correlation of numerical features with price, in descending order
kc_house_data.corr()['price'].sort_values(ascending=False)

price            1.000000
sqft_living      0.608616
bathrooms        0.480337
sqft_patio       0.313789
bedrooms         0.288954
sqft_garage      0.263674
sqft_basement    0.245005
floors           0.180589
yr_built         0.095796
sqft_lot         0.086550
yr_renovated     0.085023
id              -0.034096
Name: price, dtype: float64

From the above output we see that square feet of living space is the feature most highly correlated with price.  
Let's view a sample of the data for that feature plotted against price:

In [49]:
# plot sqft_living vs. price
kc_house_data.plot.scatter(x='sqft_living', y='price', alpha=.5, label='homes')
plt.title('Living Area vs. Price')

# add best-fit line to the plot
a, b = np.polyfit(kc_house_data['sqft_living'], kc_house_data['price'], 1)
plt.plot(kc_house_data['sqft_living'], a*kc_house_data['sqft_living']+b, color= 'lightgreen', label='best-fit line')
plt.legend();

Unsurprisingly, there is a highly visible correlation between the predictor of square-feet of living space & home sale price as indicated by the upward slope of the mass of data points.

To get an acurate understanding, let's model this with linear regression:

In [9]:
# set variables for the model
X_baseline = kc_house_data['sqft_living']
y_ = kc_house_data['price']

# create model
baseline_model = sm.OLS(y, sm.add_constant(X_baseline))
baseline_results = model.fit()

In [10]:
# view summary of the results
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.370
Method:                 Least Squares   F-statistic:                 1.771e+04
Date:                Sun, 19 Mar 2023   Prob (F-statistic):               0.00
Time:                        22:26:02   Log-Likelihood:            -4.4847e+05
No. Observations:               30111   AIC:                         8.969e+05
Df Residuals:                   30109   BIC:                         8.970e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -7.559e+04   9798.815     -7.714      0.000   -9.48e+04   -5.64e+04
sqft_living   560.5134      4.211    133.096      0.000     552.259     568.768
==============================================================================
Omnibus:                    43384.874   Durbin-Watson:                   1.862
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         47187801.072
Skew:                           8.195   Prob(JB):                         0.00
Kurtosis:                     196.242   Cond. No.                     5.56e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.56e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Model Evaluation
The overall model is statistically significant with a p-value of (nearly) 0.
The R-squared value indicates that 37% of the variance in the data is explained by the sqft_living feature.  


### Coefficients Interpetation:
The coefficients for the constant and the sqft_living feature have p-values well below an alpha of 0.05, indicating statistical significance.

* A home with 0 square-feet of living space would have a sale price of $-75,590.

* The price of a home increases by $561 per each additional square-foot of living space.

## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:
